In [ ]:
%iam_role arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role

In [ ]:
%region us-west-2

In [ ]:
%additional_python_modules "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl"
%number_of_workers 2
%connections dev-glue-redshift-connection
#%idle_timeout 30

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
#download configuration file
import boto3
session = boto3.Session()

s3 = session.resource('s3')
s3.Bucket('dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o').download_file('confs/filevine/sstm.yaml', 'sstm.yaml')

In [ ]:
with open("sstm.yaml", "r") as f:
    sstm_config = f.read()
    

In [ ]:
def readFromFile(ext, path) :
    fileContent = spark.read.format(ext).option("header", "true").option("mergeSchema", "true").load(path)
    return fileContent


In [ ]:
def writeToFile(df_join , path, mode) :
    df_join.write.parquet(path,mode=mode)

In [ ]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")


In [ ]:
from sstm_transformation.instagram_builder import InstagramBuilder

In [ ]:
#Read All dfs 

df_age =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/age.parquet")
df_city =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/city.parquet")
df_country =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/country.parquet")
df_date =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/date.parquet")
df_gender =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/gender.parquet")
df_hashtag =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/hashtag.parquet")
df_hour =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/hour.parquet")
df_locale =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/locale.parquet")
df_mediaType =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/mediaType.parquet")
df_post =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/post.parquet")


In [ ]:

builder = InstagramBuilder("sstm.yaml", spark=spark)


In [ ]:
#Upload data on S3 
from awsglue.dynamicframe import DynamicFrame
def upload_result(result):
    temp_bucket = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp"
    for key, df in result.items():
        print(f"Writing {key}")
        writeToFile(df,"s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/{}/".format(key.lower()), "overwrite")
        dyf = DynamicFrame.fromDF(dataframe=df, glue_ctx=glueContext, name=key)
        table_name = key.lower()
        dyf.show(10)
        
        datasink = glueContext.write_dynamic_frame.from_jdbc_conf(
                        frame=dyf,
                        catalog_connection="dev-glue-redshift-connection",
                        connection_options={
                            "preactions" : "delete from tsm.{}".format(table_name),
                            "dbtable": "tsm.{}".format(table_name), 
                            "database": "truve",
                        },
                        redshift_tmp_dir=temp_bucket,
                        transformation_ctx="datasink",
                    )
        '''
        #break
        redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                        "user": "truveadmin", 
                        "password": "XXXXXXX",
                        "preactions" : "delete from tsm.{}".format(table_name),
                        "dbtable": "tsm.{}".format(table_name), 
                        "redshiftTmpDir": temp_bucket} 
        datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf,
                                                        connection_type="redshift",
                                                        connection_options=redshift_connection_options)
        '''
        #break
    

In [ ]:
#Age Transformation
def age_transformation(input_df):
    #Read Raw Data
    result = builder.build_age(df=input_df)
    #print(result)
    upload_result(result)
    
age_transformation(df_age)

In [ ]:
#Age Transformation
def cities_transformation(input_df):
    #Read Raw Data
    result = builder.build_cities(df=input_df)
    #print(result)
    upload_result(result)
    
cities_transformation(df_city)

In [ ]:
#Age Transformation
def country_transformation(input_df):
    #Read Raw Data
    result = builder.build_countries(df=input_df)
    #print(result)
    upload_result(result)
    
country_transformation(df_country)

In [ ]:
#Age Transformation
def date_transformation(input_df):
    #Read Raw Data
    result = builder.build_dates(df=input_df)
    #print(result)
    upload_result(result)
    
date_transformation(df_date)

In [ ]:
#Age Transformation
def gender_transformation(input_df):
    #Read Raw Data
    result = builder.build_gender(df=input_df)
    #print(result)
    upload_result(result)
    
gender_transformation(df_gender)

In [ ]:
#Age Transformation
def post_transformation(input_df):
    #Read Raw Data
    result = builder.build_posts(df=input_df)
    #print(result)
    upload_result(result)
    
post_transformation(df_post)

In [ ]:
#Age Transformation
def hashtag_transformation(input_df):
    #Read Raw Data
    result = builder.build_hashtags(df=input_df)
    #print(result)
    upload_result(result)
    
hashtag_transformation(df_hashtag)

In [ ]:
#Age Transformation
def hour_transformation(input_df):
    #Read Raw Data
    result = builder.build_hours(df=input_df)
    #print(result)
    upload_result(result)
    
hour_transformation(df_hour)

In [ ]:
#Age Transformation
def locale_transformation(input_df):
    #Read Raw Data
    result = builder.build_locals(df=input_df)
    #print(result)
    upload_result(result)
    
locale_transformation(df_locale)

In [ ]:
#Age Transformation
def mediatype_transformation(input_df):
    #Read Raw Data
    result = builder.build_mediatypes(df=input_df)
    #print(result)
    upload_result(result)
    
mediatype_transformation(df_mediaType)

In [ ]:
%stop_session